In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [4]:
data=pd.read_csv('Data/Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [7]:
#encode categorical value
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [8]:
#one hot encode 'Geography'ArithmeticError
encode_geo=OneHotEncoder(handle_unknown='ignore')
geo = encode_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo,columns=encode_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [ ]:
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)

KeyError: "['Geography'] not found in axis"

In [12]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [14]:
#Split data into features and target
X=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']

In [19]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [21]:
with open('Data/label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('Data/encode_geo.pkl','wb') as file:
    pickle.dump(encode_geo,file)

with open('Data/scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN regression Porblem

In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [25]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1) #defualt liner actiation function
])

model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

log_dir="regressionlogs/fit"+datetime.datetime.now().strftime("%d/%m/%Y, %H:%M:%S")
tensorboard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [27]:
early_stopping_callacks=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [29]:
#Train The model
history=model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[early_stopping_callacks,tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 802us/step - loss: 99887.9922 - mae: 99887.9922 - val_loss: 98412.7031 - val_mae: 98412.7031
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 591us/step - loss: 98564.1641 - mae: 98564.1641 - val_loss: 96088.3906 - val_mae: 96088.3906
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 495us/step - loss: 96061.2344 - mae: 96061.2344 - val_loss: 90317.4922 - val_mae: 90317.4922
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 492us/step - loss: 89174.8438 - mae: 89174.8438 - val_loss: 81300.1016 - val_mae: 81300.1016
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 492us/step - loss: 79843.8438 - mae: 79843.8438 - val_loss: 70902.6797 - val_mae: 70902.6797
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 488us/step - loss: 69228.2734 - mae: 69228.2734 - val_loss: 61772.3906 - val_mae: 61772.3906
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 486us/step - loss: 60619.3516 - mae: 60619.3516 - val_loss: 55530.7070 - val_mae: 55530.7070
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 

In [30]:
%load_ext tensorboard

In [35]:
%tensorboard --logdir regressionlogs/fit31/12

Reusing TensorBoard on port 6006 (pid 75833), started 0:00:45 ago. (Use '!kill 75833' to kill it.)

In [36]:
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE: {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 422us/step - loss: 51069.8555 - mae: 51069.8555
Test MAE: 50169.4453125


In [37]:
model.save('regressionmodel.h5')